In [1]:
import glob
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize

In [2]:
# # Need to download this if first time using nltk
# nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/pranavgoel/miniconda3/envs/pg3/lib/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
DATA_PATH = '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/prior_data_apr_jun_2023/'

In [4]:
all_csv_paths=glob.glob(DATA_PATH+'*texts_and_info.csv*')
all_csv_paths

['/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/prior_data_apr_jun_2023/nytimes_foxnews_article_texts_and_info.csv',
 '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/prior_data_apr_jun_2023/california_article_texts_and_info.csv',
 '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/prior_data_apr_jun_2023/texas_article_texts_and_info.csv',
 '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/prior_data_apr_jun_2023/illinois_article_texts_and_info.csv',
 '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/prior_data_apr_jun_2023/ohio_article_texts_and_info.csv',
 '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/prior_data_apr_jun_2023/florida_article_texts_and_info.csv',
 '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/prior_data_apr_jun_2023/newyork_article_texts_and_info.csv']

In [5]:
def load_and_convert(path):
    '''load one dataset from the path, drop the na, and convert date to timestamp'''
    df= pd.read_csv(path)
    # drop na of text field, right?
    df=df[df['text'].notna()]
    # convert to datetime so i can idmax
    return df.assign(publish_date=pd.to_datetime(df['publish_date']))

In [6]:
def drop_duplicates(df, columns=['url','title','text'], keep_col='publish_date', ignore_case=True):
    '''Check if either of the column in the input columns contains duplicates.
    If so, keep the row with the max value of the keep_col parameter. In our case, the latest timestamp.
    Because we are doing either, not all, I will need to run group by each column at a time untill exhaustion.
    '''

    if not ignore_case:
        for col in columns:
            df=df.loc[df.groupby(col)[keep_col].idxmax()]
    else:
        for col in columns:
            df=df.loc[df.groupby(df[col].str.lower())[keep_col].idxmax()]
 
    return df.sort_index()

In [119]:
# def get_sent_count(text):
#     '''return the sentence count of a text'''

#     return len(sent_tokenize(text))

In [7]:
all_csv_paths[:1]

['/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/prior_data_apr_jun_2023/nytimes_foxnews_article_texts_and_info.csv']

In [8]:
for file_pat in all_csv_paths[:1]:
    temp_df= load_and_convert(file_pat)
    print(f" loading {file_pat} with {len(temp_df)} rows")
    temp_df_dedup=drop_duplicates(temp_df, ignore_case=True)
    print(f"de-duped! remaining rows {len(temp_df_dedup)}")

    temp_df_dedup['sent_count']=temp_df_dedup['text'].map(lambda text: len(sent_tokenize(text)))
    print(temp_df_dedup['sent_count'].describe())
    temp_df_dedup_drop_short_text=temp_df_dedup[temp_df_dedup['sent_count']>5]

    print(f"Final row counts: {len(temp_df_dedup_drop_short_text)}")

    new_path=file_pat.split('.csv')[0]+'_dedup.csv'
    # new_path=f"/home/crazybrokeasian/projects/{file_pat.split('/')[-1].split('.csv')[0]}_dedup.csv"
    print(f'saving new file to {new_path}')
    
    temp_df_dedup_drop_short_text.to_csv(new_path, index=False)

    print('--------')


 loading /home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/prior_data_apr_jun_2023/nytimes_foxnews_article_texts_and_info.csv with 2344 rows
de-duped! remaining rows 1655
count    1655.000000
mean       26.329909
std        38.993751
min         2.000000
25%        14.000000
50%        21.000000
75%        31.000000
max       827.000000
Name: sent_count, dtype: float64
Final row counts: 1580
saving new file to /home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/prior_data_apr_jun_2023/nytimes_foxnews_article_texts_and_info_dedup.csv
--------


---

Below is prior code working with all the data before foxnews fix -- it still applies to all state media collections. Just de-duping the nytimes_foxnews file since that's the only thing changing after the foxnews fix.

In [10]:
for file_pat in all_csv_paths:
    temp_df= load_and_convert(file_pat)
    print(f" loading {file_pat} with {len(temp_df)} rows")
    temp_df_dedup=drop_duplicates(temp_df, ignore_case=True)
    print(f"de-duped! remaining rows {len(temp_df_dedup)}")

    temp_df_dedup['sent_count']=temp_df_dedup['text'].map(lambda text: len(sent_tokenize(text)))
    print(temp_df_dedup['sent_count'].describe())
    temp_df_dedup_drop_short_text=temp_df_dedup[temp_df_dedup['sent_count']>5]

    print(f"Final row counts: {len(temp_df_dedup_drop_short_text)}")

    new_path=file_pat.split('.csv')[0]+'_dedup.csv'
    # new_path=f"/home/crazybrokeasian/projects/{file_pat.split('/')[-1].split('.csv')[0]}_dedup.csv"
    print(f'saving new file to {new_path}')
    
    temp_df_dedup_drop_short_text.to_csv(new_path, index=False)

    print('--------')


 loading /home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/nytimes_foxnews_article_texts_and_info.csv with 3479 rows
de-duped! remaining rows 3213
count    3213.000000
mean       25.718332
std        38.975845
min         1.000000
25%        12.000000
50%        20.000000
75%        29.000000
max       787.000000
Name: sent_count, dtype: float64
Final row counts: 3066
saving new file to /home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/nytimes_foxnews_article_texts_and_info_dedup.csv
--------
 loading /home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/california_article_texts_and_info.csv with 14450 rows
de-duped! remaining rows 8476
count    8476.000000
mean       45.690066
std        55.563920
min         1.000000
25%        19.000000
50%        33.000000
75%        54.000000
max       834.000000
Name: sent_count, dtype: float64
Final row counts: 8059
saving new file to /home/pranavgoel/trans-fer-entropy/obtaining_news_collections/dat

# Data Profiling

In [5]:
all_de_dup=glob.glob(DATA_PATH+'*_dedup.csv')
all_de_dup

['/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/newyork_article_texts_and_info_dedup.csv',
 '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/nytimes_foxnews_article_texts_and_info_dedup.csv',
 '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/florida_article_texts_and_info_dedup.csv',
 '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/illinois_article_texts_and_info_dedup.csv',
 '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/texas_article_texts_and_info_dedup.csv',
 '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/california_article_texts_and_info_dedup.csv',
 '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/ohio_article_texts_and_info_dedup.csv']

In [6]:
for de_dup in all_de_dup:
    print(de_dup)
    temp_df=pd.read_csv(de_dup)
    print(len(temp_df))
    print(temp_df['media_name'].nunique())
    print(temp_df['publish_date'].max(), temp_df['publish_date'].min())


/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/newyork_article_texts_and_info_dedup.csv
2910
129
2023-06-30 22:42:34.000000 2023-04-01 04:44:31.000000
/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/nytimes_foxnews_article_texts_and_info_dedup.csv
944
2
2023-06-30 21:00:21 2023-04-01 09:00:07
/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/florida_article_texts_and_info_dedup.csv
1283
72
2023-06-30 23:57:00 2023-04-01 18:00:40
/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/illinois_article_texts_and_info_dedup.csv
692
63
2023-06-30 17:50:51 2023-04-01 13:06:27
/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/texas_article_texts_and_info_dedup.csv
1816
98
2023-06-30 22:26:57 2023-04-01 05:01:05
/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/california_article_texts_and_info_dedup.csv
7571
340
2023-06-30 22:43:00.000000 2023-04-01 00:46:56.000000
/home/pranavgoel/trans-fer-

In [7]:
pd.read_csv('/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/nytimes_foxnews_article_texts_and_info_dedup.csv')

,media_name,publish_date,title,url,subtitle,text,sent_count
0,New York Times,2023-04-01 09:00:07,What Decades of Providing Trans Health Care Ha...,https://www.nytimes.com/2023/04/01/opinion/tra...,The field of transgender medicine is every bit...,Decades of medical experience and research sin...,17
1,New York Times,2023-04-02 14:37:17,Trump Flourishes in the Glare of His Indictment,https://www.nytimes.com/2023/04/02/us/politics...,The former president’s appetite for attention ...,But those were different times and different p...,10
2,New York Times,2023-04-03 10:42:21,Biden’s Reluctant Approach to Free Trade Draws...,https://www.nytimes.com/2023/04/03/us/politics...,A law intended to bolster clean energy manufac...,Those provisions have angered allies in Europe...,8
3,New York Times,2023-04-03 13:26:57,Yeshiva University’s Ban on L.G.B.T.Q. Club Le...,https://www.nytimes.com/2023/04/03/nyregion/ye...,A lawmaker asked inspectors to look at million...,A state inspector has been asked to review whe...,8
4,New York Times,2023-04-04 13:11:40,"New Zealand’s Soccer Team to Wear Dark Shorts,...",https://www.nytimes.com/2023/04/04/sports/socc...,The women’s soccer team said its players would...,"For the first time, New Zealand’s women’s socc...",6
...,...,...,...,...,...,...,...
939,Fox News,2023-06-30 16:39:08,University demands professor learn free speech...,https://www.foxnews.com/media/university-deman...,The University of Cincinnati reprimanded a pro...,The University of Cincinnati demanded adjunct ...,19
940,New York Times,2023-06-30 16:17:50,These States Have Anti-discrimination Laws Pro...,https://www.nytimes.com/2023/06/30/us/politics...,The laws explicitly protect people from being ...,Federal law protects Americans from being disc...,6
941,Fox News,2023-06-30 17:56:39,Texas Rep Wesley Hunt challenges 'woke' corpor...,https://www.foxnews.com/politics/texas-rep-wes...,"Rep. Wesley Hunt, R-Texas, introduced a resolu...","Rep. Wesley Hunt, R-Texas, is challenging Pres...",18
942,New York Times,2023-06-30 19:47:45,"Before the hypothetical gay wedding case, he w...",https://www.nytimes.com/live/2023/06/30/us/gay...,NaN,Pinned\n\nLorie Smith said her Christian faith...,63


In [16]:
all_dedup_files=glob.glob('/home/pranavgoel/trans-fer-entropy/article_filtering/transgender_filtered_data_files/*.csv')

In [6]:
test_fox=pd.read_csv('/home/pranavgoel/trans-fer-entropy/article_filtering/transgender_filtered_data_files/nytimes_foxnews_article_texts_and_info_dedup.csv')

In [9]:
test_fox[test_fox['url'].str.contains('houghton')]

,domain,publish_date,title,url,subtitle,text,sent_count,gender_label
713,nytimes.com,2023-05-19 05:00:35,A Small NY University Fired Employees For Usin...,https://www.nytimes.com/2023/05/19/nyregion/ho...,The firings set off a debate at Houghton Unive...,When Raegan Zelaya and Shua Wilmot decided to ...,7,1


In [15]:
test_fox[test_fox.url.str.strip().str.lower().duplicated()]

,domain,publish_date,title,url,subtitle,text,sent_count,gender_label


In [20]:
for temp_dedup in all_dedup_files:
    temp_dedup_file=pd.read_csv(temp_dedup)
    print(temp_dedup)
    print(f"dup urls: {temp_dedup_file[temp_dedup_file.url.str.strip().str.lower().duplicated()]}")
    print(f"dup titles: {temp_dedup_file[temp_dedup_file.title.str.strip().str.lower().duplicated()]}")
    print(f"dup body text: {temp_dedup_file[temp_dedup_file.text.str.strip().str.lower().duplicated()]}")
    print('----')
 


/home/pranavgoel/trans-fer-entropy/article_filtering/transgender_filtered_data_files/ohio_article_texts_and_info_dedup_without_national_outlets.csv
dup urls: Empty DataFrame
Columns: [Unnamed: 0, media_name, publish_date, title, url, subtitle, text, sent_count, domain, gender_label]
Index: []
dup titles: Empty DataFrame
Columns: [Unnamed: 0, media_name, publish_date, title, url, subtitle, text, sent_count, domain, gender_label]
Index: []
dup body text: Empty DataFrame
Columns: [Unnamed: 0, media_name, publish_date, title, url, subtitle, text, sent_count, domain, gender_label]
Index: []
----
/home/pranavgoel/trans-fer-entropy/article_filtering/transgender_filtered_data_files/nytimes_foxnews_article_texts_and_info_dedup.csv
dup urls: Empty DataFrame
Columns: [domain, publish_date, title, url, subtitle, text, sent_count, gender_label]
Index: []
dup titles: Empty DataFrame
Columns: [domain, publish_date, title, url, subtitle, text, sent_count, gender_label]
Index: []
dup body text: Empty D